In [ ]:
from __future__ import division

import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
import matplotlib.patheffects as PathEffects
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import sys

from svgpathtools import parse_path



import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")




## Setting up and creating dataframe for analysis


In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
features_dir= os.path.join(results_dir,'features')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))

if not os.path.exists(results_dir):
    os.makedirs(results_dir)  

if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)  

if not os.path.exists(features_dir):
    os.makedirs(features_dir)
    
## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))        
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

### setting up connection to mongo

In [ ]:
#### set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
key  = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['semantic_parts']
coll = db['sketchpad_basic']

# which iteration name should we use?
iterationName = 'pilot0'

In [ ]:
num_sketches = coll.find({'iterationName':iterationName}).count()
print 'We have {} annotations so far.'.format(num_sketches)

In [ ]:
jefan = ['A1MMCS8S8CTWKU','A1MMCS8S8CTWKV','A1MMCS8S8CTWKS']
hawkrobe = ['A1BOIDKD33QSDK']
kmukherjee = ['A1WU4IHJNQGVAY']
researchers = jefan + hawkrobe  + kmukherjee

In [ ]:
unique_assignments = coll.find({'iterationName':iterationName}).distinct('aID')
print 'We have had {} unique sessions'.format( len(unique_assignments))

In [ ]:
## get list of unique_assignments
unique_assignments = coll.find({'iterationName':iterationName}).distinct('aID')

### initialize a bunch of stuff
orig_gameID = [] # the gameID from which this sketch was sourced
outcome =[] #original outcome for that trial- true/false
orig_trial_num = [] # the trialnum in the original game from which this sketch was sourced -- 
sketch_id = [] # concatenation of orig_gameID and orig_trial_num -- 
assignmentID = [] # the session in which this annotation was collected -- 
annotation_id = [] # the unique ID for each annotation trial (different for each session the same sketch appears in)
category = [] # e.g., "chair"
target = [] # e.g., "inlay"
condition = [] # e.g., "closer" vs. "further" or "repeated" vs. "control
trial_num = [] 
workerID = [] #mTurk workerId
spline_id =[] #unique spline identifier
time_submitted = [] # when the participant clicked "next sketch"
time_labeled = [] # unique to each spline labeled
time_clicked = [] # when this spline was clicked/selected
num_strokes_in_sketch = [] # how many strokes in this sketch
num_splines_in_sketch = [] # how many spline elements in this sketch
stroke_num = [] # which stroke number this labeled spline came from
cumulative_spline_num = [] # spline index in the cumulative spline sequence for the entire sketch
within_stroke_spline_num = [] # spline index for the current stroke
cumulative_bout_num= [] #which bout of annotation the spline belonged to
part_bout_num =[] #which part-specific bout of annotation the spline belonged to
label = [] # the label provided by the participant
spline_svg_string = [] # the svg spline string that earned this label
sketch_svg_string = [] # the entire svg string correponding to this sketch
annotation_flag = [] # this is True if all splines were labeled as the same thing
annotation_spline_id = [] #unique identifier for specific annotation of a spline
png=[] #png string for the annotated sketch
stroke_id=[]

## loop through all the unique assignments that have submitted things
for this_assignment, aID in enumerate(unique_assignments):
    if this_assignment%100==0:
        print 'Analyzing sketches from assignment {} of {}  ...'.format(this_assignment, len(unique_assignments))

    ### get all the sketch recs for this assignment
    sketch_recs = coll.find({'$and': [{'iterationName':iterationName}, {'aID':aID}]}).sort('time')

    try:

        for sketch_ind,sketch in enumerate(sketch_recs):
            ## get annotations embedded within record
            sketch_cat = sketch['category']
            annotations_string = sketch['annotations']
    
            ## convert to json dictionary
            _annotations_dict = json.loads(annotations_string)  
           
            annotations_dict = _annotations_dict[0][sketch_cat]
            png_string = _annotations_dict[0]['png']
            num_splines = len(annotations_dict)
            for annotation in annotations_dict:
                assert sketch['numSplines']==num_splines                
                ## get spline-level metadata
            
                workerID.append(h.encode(key,sketch['wID']))
                label.append(annotation['label'])
                stroke_num.append(annotation['strokeNum'])
                spline_svg_string.append(annotation['svgString'])
                cumulative_spline_num.append(annotation['cumulativeSplineNum'])
                within_stroke_spline_num.append(annotation['withinStrokeSplineNum'])
                time_clicked.append(annotation['timeClicked'])
                time_labeled.append(annotation['timeLabeled'])
                spline_id.append('{}_{}_{}'.format(sketch['originalGameID'],sketch['originalTrialNum'],annotation['cumulativeSplineNum']))
                stroke_id.append('{}_{}_{}'.format(sketch['originalGameID'],sketch['originalTrialNum'],annotation['strokeNum']))
                cumulative_bout_num.append(annotation['boutNum'])
                part_bout_num.append(annotation['partBoutNum'])
                ## get sketch-level metadata
                orig_gameID.append(sketch['originalGameID'])   
                outcome.append(sketch['originalOutcome'])
                orig_trial_num.append(sketch['originalTrialNum'])
                sketch_id.append('{}_{}'.format(sketch['originalGameID'],sketch['originalTrialNum']))
                annotation_id.append('{}_{}_{}'.format(sketch['originalGameID'],sketch['originalTrialNum'],sketch['aID']))
                assignmentID.append(sketch['aID'])
                category.append(sketch['category'])
                target.append(sketch['target'])
                png.append(png_string)
                condition.append(sketch['condition'])
                time_submitted.append(sketch['time'])
                trial_num.append(sketch['trialNum'])
                num_splines_in_sketch.append(sketch['numSplines'])
                num_strokes_in_sketch.append(sketch['numStrokes'])
                sketch_svg_string.append(sketch['svg'])
                annotation_flag.append(sketch['sameAnnotflag'])
                annotation_spline_id.append('{}_{}_{}_{}'.format(sketch['originalGameID'],sketch['originalTrialNum'],sketch['aID'],annotation['cumulativeSplineNum']))
                
    except AssertionError:
        print 'There were unequal numbers for sketch["numSplines"] vs. num_splines for sketch {} from {}'.\
                format(sketch['trialNum'], sketch['aID'])


In [ ]:
## make group dataframe 
D = pd.DataFrame([workerID,orig_gameID, orig_trial_num, outcome, sketch_id, category, assignmentID,  target, \
                  annotation_id, condition, trial_num, time_submitted,\
                 time_labeled, time_clicked, num_strokes_in_sketch, num_splines_in_sketch,\
                 stroke_num, cumulative_spline_num, within_stroke_spline_num, cumulative_bout_num,\
                 part_bout_num, label, spline_svg_string, sketch_svg_string, spline_id, stroke_id,\
                  annotation_spline_id,png])
D = D.transpose()
D.columns = ['workerID','orig_gameID', 'orig_trial_num','outcome', 'sketch_id', 'category', 'assignmentID', 'target',\
             'annotation_id', 'condition', 'trial_num', 'time_submitted',\
             'time_labeled', 'time_clicked', 'num_strokes_in_sketch', 'num_splines_in_sketch',\
             'stroke_num', 'cumulative_spline_num', 'within_stroke_spline_num', 'cumulative_bout_num', 'part_bout_num', 'label',\
             'spline_svg_string', 'sketch_svg_string', 'spline_id','stroke_id','annotation_spline_id','png']
D=D[D['assignmentID']!='']


print 'Annotations dataframe contains {} rows and {} columns.'.format(D.shape[0],D.shape[1])

In [ ]:
##Check to see what dataframe looks like
D.head()

In [ ]:
###Changing the NAs to "None" strings

for ind, row in D.iterrows():
    if row['label'] is None:
        row['label'] = "None"

In [ ]:
#Get a count of how many unique sketches have been annotated

unique_sketches = np.unique(D['sketch_id'].values)
print 'We have {} unique sketches.'.format(len(unique_sketches))

In [ ]:
###How many unique annotations do we have in total?
len(D['annotation_id'].unique())

In [ ]:
###Removing any annotations that don't have all splines annotated

for this_sketch in unique_sketches:
    DS=D[D['sketch_id']==this_sketch]
    for this_annot in np.unique(DS['annotation_id']):
        DSS= DS[DS['annotation_id']==this_annot]
        if DSS[DSS['label']== 'None'].shape[0]>0:
            D=D[D['annotation_id']!=this_annot]
        

In [ ]:
##How many annotations after filtering?
len(D['annotation_id'].unique())

# Visualizations and desriptive statistics 

## Number of annotations per sketch

In [ ]:
## get number of times each sketch has been annotated
num_times_annotated = []
for this_sketch_id in unique_sketches:
    num_times_annotated.append(D[D['sketch_id']==this_sketch_id]['assignmentID'].nunique())
    
## make a histogram
sns.set_context('talk')
plt.figure(figsize=(6,5))
h = plt.hist(num_times_annotated)
plt.xticks(np.arange(0, 4, step=1))
plt.title('Times each sketch has been annotated')
plt.ylabel('number of sketches')


In [ ]:
###Subesttting for sketches that have been annotated 3 times
num_annots=3
##Why are some assignment IDs blank?
D=D[D['assignmentID']!='']
for this_sketch_id in unique_sketches:
    if D[D['sketch_id']==this_sketch_id]['assignmentID'].nunique()!=num_annots:
        D=D[D['sketch_id']!=this_sketch_id]
unique_sketches = np.unique(D['sketch_id'].values)

In [ ]:
##How many sketches do we have with 3 annotations?
len(np.unique(D.sketch_id))

## Looking at time taken to annotate

In [ ]:
##Make sure the number of splines for each sketch is consistent across annotations
for this_sketch in unique_sketches:
    assert len(np.unique(D[D['sketch_id']==this_sketch]['num_splines_in_sketch'].values))==1

unique_annotation_trials = np.unique(D['annotation_id'].values)
    

In [ ]:
## get annotation time for each annotation trial
annotation_time = []
spline_number_in_sketch = []

for this_annotation_trial in unique_annotation_trials:
    earliest_click = float(np.min(D[D['annotation_id']==this_annotation_trial]['time_clicked']))
    ## all of the splines were submitted at the same time, so time_submitted should be identical for all splines in an annotation trial
    assert len(np.unique(D[D['annotation_id']==this_annotation_trial]['time_submitted'].values))==1
    final_submission = np.unique(D[D['annotation_id']==this_annotation_trial]['time_submitted'].values)[0]
    annotation_time.append(final_submission-earliest_click)        
    if np.isnan(final_submission) or np.isnan(earliest_click):
        print 'One of these timestamps is a NaN. Probably means that the participant skipped this trial:'
        print this_annotation_trial
        print 'final_submission: {}, earliest_click: {}'.format(final_submission, earliest_click)
    #### then we will extract how "complex" each sketch i
    assert len(np.unique(D[D['annotation_id']==this_annotation_trial]['num_splines_in_sketch'].values))==1    
    spline_number_in_sketch.append(np.unique(D[D['annotation_id']==this_annotation_trial]['num_splines_in_sketch'])[0])
    

In [ ]:
## convert annotation time to seconds
annotation_time_seconds = np.array(annotation_time)/1000

## make dataframe with annotation time and spline number
unique_annotation_trials, spline_number_in_sketch, annotation_time_seconds = map(list, [unique_annotation_trials, spline_number_in_sketch, annotation_time_seconds])
T = pd.DataFrame([unique_annotation_trials,spline_number_in_sketch,annotation_time_seconds])
T = T.transpose()
T.columns = ['annotation_trial','spline_number_in_sketch','annotation_time']

## some preprocessing of T
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)


## make numeric types
T = h.convert_numeric(T,'spline_number_in_sketch')
T = h.convert_numeric(T,'annotation_time')

## also remove the skipped trial where annotation time is a NaN
T = T[~np.isnan(T['annotation_time'])]

In [ ]:
## make scatterplot of relationship between annotation time and spline number
plt.figure(figsize=(6,6))
sns.scatterplot(x='annotation_time',
                y='spline_number_in_sketch',
                data=T)
plt.ylabel('number of splines in sketch')
plt.xlabel('annotation time (s)')
plt.title('Do more complex sketches take longer to annotate?')
plt.xlim(0,60*5) ## 5 minute cutoff ...

In [ ]:
r, p = stats.spearmanr(T['annotation_time'],T['spline_number_in_sketch'])
print 'Spearman correlation between annotation time and spline number in sketch' 
print 'r = {}, p = {}'.format(r,p)

## Looking at part occurrence 

In [ ]:
## get the list of unique labels applied to sketches
unique_labels = np.unique(D.label.values)

## Removing Nones and obviously wrong super long lables
unique_labels = [i for i in unique_labels if i is not None]
unique_labels = [i for i in unique_labels if len(i)<900]

print 'we have {} unique labels'.format(len( unique_labels))

In [ ]:
##Get a list of categories
unique_cats = np.unique(D['category'])

##Create empty dictionary with categories as keys. We will use this to store part occurrence data for our categories
label_vect_dict = {unique_cats[0]:None,unique_cats[1]:None,unique_cats[2]:None,unique_cats[3]:None}

In [ ]:
##Create vectors that contain the number of part instances in each sketch

for category in unique_cats:
    DS= D[D['category']==category]
    unique_sketches_in_cat = np.unique(DS['sketch_id'])
    unique_labels_in_cat = np.unique(DS['label'])
    ## initialize matrix that has the correct dimensions
    Label_Vec = np.zeros((len(unique_sketches_in_cat),len(unique_labels_in_cat)), dtype=int)
    unique_labels_in_cat= np.array(unique_labels_in_cat)
    for s,this_sketch in enumerate(unique_sketches_in_cat):
        label_vec = np.zeros(len(unique_labels_in_cat),dtype=int)
        DSS = DS[DS['sketch_id']==this_sketch]
        annotation_ids = np.unique(DSS['annotation_id'].values)    
        for this_annotation in annotation_ids:
            DSA = DSS[DSS['annotation_id']==this_annotation]
            label_list = DSA.label.values
            for this_label in label_list:
                label_ind = unique_labels_in_cat==this_label
                label_vec[label_ind] += 1
            
        Label_Vec[s,:]=label_vec/num_annots
    label_vect_dict[category]= Label_Vec

In [ ]:
D[D['category']=='car'].label.unique()

In [ ]:
maplist={}
maplist
{'body':['59 decal','Body and hood','Body and windshield','Gas Cap', 'Gas tank','Logo','Number','Number Decal','logo','grill',\
                'Grille','Grill','hubcap','seat','grille','ROOF','number','59 decal'],
 'bumper':[],
 'door':['DOOR HANDLE'],
 'headlight':[],
 'hood':[],
 'trunk':[],
 'unknown':['Letter R','Letter e','Letter D','Says the word Drive'],
 'wheel':['rim','Tire',],
 'window':[],
 'windshield':[]
}
maplist[]

# D[D['label'] in maplist, 'label']='body'


In [ ]:
valid_labels_dict['car']

In [ ]:
valid_labels=[]
valid_labels_dict={}
for category in unique_cats:
    vect = label_vect_dict[category]
    thresh = 50
    #print 'These are the labels that appear at least {} times:'.format(thresh)
    #print unique_labels[np.sum(Label_Vec,0)>thresh]
    unique_labels_in_cat = np.unique(D[D['category']==category]['label'])
    plot_labels= unique_labels_in_cat[np.sum(vect,0)>thresh]
    valid_labels_dict[category]=plot_labels
    valid_labels.append(plot_labels)


    prop_labels=[]
    for part in plot_labels:
        DS=D[D['category']==category]
        prop_labels.append(DS[DS['label']==part]['annotation_id'].nunique()/DS['annotation_id'].nunique())
    
    
    sns.set_context('talk')
    plt.figure(figsize=(12,7))
    plt.ylim(0,1)
    h = plt.bar(plot_labels,prop_labels)
    plt.title('Proportion of {} annotations with labels'.format(category))
    plt.ylabel('proportion of annotations')
    plt.xlabel('Part')

In [ ]:
##flattening valid labels
valid_labels = [item for sublist in valid_labels for item in sublist]


In [ ]:
len(np.unique(valid_labels))

In [ ]:
##Creating a stroke-level dataframe that takes the mode value of annotation for its children splines to set as its
##label value

from collections import Counter


from collections import OrderedDict
stroke_svgs=OrderedDict()
for category in unique_cats:
    DS=D[D['category']==category]
    for sketch in np.unique(DS['sketch_id']):
        DSS=DS[DS['sketch_id']==sketch]
        for stroke in np.unique(DSS['stroke_num']):
            DSA=DSS[DSS['stroke_num']==stroke]
            DSA=DSA.reset_index()
            stroke_svgs[DSA['stroke_id'][0]] = DSA['sketch_svg_string'][0][stroke]

            
            
stroke_svg_df= pd.DataFrame.from_dict(stroke_svgs, orient='index')    
stroke_group_data= D.groupby('stroke_id').agg(lambda x: Counter(x).most_common(1)[0][0])
labels= pd.DataFrame(stroke_group_data[['sketch_id','label','stroke_num','condition','target','category','outcome']])
stroke_df=pd.merge(stroke_svg_df,labels,left_index=True, right_index =True)
stroke_df.reset_index(level=0, inplace=True)
stroke_df=stroke_df.rename(index=str, columns={"index": "stroke_id", 0: "svg"})


In [ ]:
##Adding total arclength information to stroke dataframe

from svgpathtools import parse_path
import svgpathtools

stroke_df['arc_length'] = ""
for s,stroke in stroke_df.iterrows():
    try:
        stroke_df['arc_length'][s] = parse_path(stroke['svg']).length()
    except ZeroDivisionError:
        print 'zero div error'
        stroke_df['arc_length'][s] = 0
   

In [ ]:
#Saving out the PNG for the sketches
run=False
if run==True:
    from matplotlib.pyplot import imshow
    import base64


    #num_diff_annots = []
    for this_sketch_id in unique_sketches:
        DS=D[D['sketch_id']==this_sketch_id]
        unique_splines = np.unique(DS['cumulative_spline_num'])
        for i,this_spline in enumerate(unique_splines):
            DSS =DS[DS['cumulative_spline_num']==this_spline]
            num_diff_annots= len(np.unique(DSS['label']))
        if num_diff_annots>0:  ##Will update this conditional once we have more annots
            for instance in np.unique(DS['annotation_id']):
                    imgdata = base64.b64decode(DS[DS['annotation_id']==instance].iloc[0]['png'])
                    filename = '{}_{}'.format(instance, DS[DS['annotation_id']==instance].iloc[0]['target'])  # I assume you have a way of picking unique filenames
                    with open(filename, 'wb') as f:
                        f.write(imgdata)
                    im = Image.open(filename)
                    plt.figure()
                    imshow(im)
                    plt.xticks([])
                    plt.yticks([])
                    #plt.savefig(xx)
run=False

## Making part-transition matrices

In [ ]:
####Setting up for building transition matrices for each different category
from itertools import product as p
run=False
if run==True:
   
    tm_dict={}
    for category in unique_cats:
        stroke_df_s = stroke_df[stroke_df['category']==category]  
        num_uniq_labs = len(np.unique(stroke_df_s['label']))
        temp_array = np.zeros([num_uniq_labs*num_uniq_labs,2],dtype='|S50')
        ind=0
        for roll in p(stroke_df_s['label'].unique().tolist(), repeat = 2):
            temp_array[ind,]= roll
            ind+=1
        for sketch in stroke_df_s.sketch_id.unique():
            sketch_df= stroke_df_s[stroke_df_s.sketch_id==sketch]
            sketch_df['incr_stroke_num'] = sketch_df['stroke_num']+1
            tm_df=sketch_df.merge(sketch_df, right_on='stroke_num', left_on='incr_stroke_num', how='inner' )
            plot_matrix_x= tm_df.label_x.append(pd.Series(temp_array[:,0]))
            plot_matrix_y=tm_df.label_y.append(pd.Series(temp_array[:,1]))
            ct_df= pd.crosstab(plot_matrix_x, plot_matrix_y)-1
            #ct_df= pd.crosstab(tm_df.label_x, tm_df.label_y)
            mat= np.matrix(ct_df).sum()
            plot_df = ct_df.div(mat, axis=0).round(2)
            plot_df=plot_df.fillna(0)
            tm_dict[sketch]= plot_df

    for category in unique_cats:
        cat_matrices=[]
        DS=D[D['category']==category]
        for sketch in DS['sketch_id'].unique():
            cat_matrices.append(tm_dict[sketch])
        fig,ax=plt.subplots(1, 1, figsize = (10, 8), dpi=150)
        agg_matrix= sum(cat_matrices)
        divider=np.matrix(agg_matrix).sum()
        agg_matrix = agg_matrix.div(divider, axis=0).round(2)
        sns.heatmap(agg_matrix,cmap="YlGnBu", annot=True)
        plt.title("Part transition matrix for {}".format(category))
        ax.set_ylabel('')    
        ax.set_xlabel('')

run=False

In [ ]:
## Getting the number of unique labels assigned to a given spline across annotations
num_diff_annots = []
for this_cat in unique_cats:
    DS=D[D['category']==this_cat]
    labels = valid_labels_dict[this_cat]
    unique_sketches_in_cat=np.unique(DS['sketch_id'])
    

   
    for this_sketch_id in unique_sketches_in_cat:
        DSA=DS[DS['sketch_id']==this_sketch_id]
        unique_splines = np.unique(DSA['cumulative_spline_num'])
        for i,this_spline in enumerate(unique_splines):
            DSB =DSA[DSA['cumulative_spline_num']==this_spline]
            numannots= 4-len(np.unique(DSB['label']))
            if numannots==0:
                numannots=1
            num_diff_annots.append(numannots)



In [ ]:
#plotting variability in spline annots
h= plt.hist(num_diff_annots, bins= range(1,5), align='left', density='True')
plt.title('Inter-annotator reliability')
plt.ylabel('proportion of splines')
plt.xlabel('Annotator agreement on label')
plt.xticks([1,2,3],['0/3','2/3','3/3'])

In [ ]:
spline_df= D.groupby('spline_id').agg(lambda x: Counter(x).most_common(1)[0][0])

In [ ]:
spline_df.reset_index(level=0, inplace=True)

In [ ]:
for this_cat in unique_cats:
    labels = valid_labels_dict[this_cat]
    DS=spline_df[spline_df['category']==this_cat]
    spline_annots_per_stroke = []
    unique_sketches_in_cat= np.unique(DS['sketch_id'])
    for this_sketch_id in unique_sketches_in_cat:
        DSA=DS[DS['sketch_id']==this_sketch_id]
        unique_strokes = np.unique(DSA['stroke_num'])
        for i,this_stroke in enumerate(unique_strokes):
            DSB =DSA[DSA['stroke_num']==this_stroke]
            numlabels= DSB['label'].nunique()
            spline_annots_per_stroke.append(numlabels)

In [ ]:
h= plt.hist(spline_annots_per_stroke, bins =range(1,8), align='left', density="True")
plt.title('Within-stroke label agreement')
plt.ylabel('proportion of strokes')
plt.xlabel('number of different labels within stroke')

In [ ]:
for this_cat in unique_cats:
    DS=stroke_df[stroke_df['category']==this_cat]
    labels= valid_labels_dict[this_cat]
    strokes_in_part_vect = np.zeros((len(np.unique(DS['sketch_id']))*len(labels),3), dtype='|a1000')
    ind=0
    for this_sketch in np.unique(DS['sketch_id']):    
        DSA= DS[DS['sketch_id']==this_sketch]
        for this_label in labels:
            DSB=DSA[DSA['label']==this_label]
            strokes_in_part_vect[ind,]=[this_sketch, this_label,len(np.unique(DSB['stroke_num']))]
            ind+=1
    strokes_in_part_vect=strokes_in_part_vect[~np.all(strokes_in_part_vect == '', axis=1)]
    strokes_in_part_df= pd.DataFrame(strokes_in_part_vect, columns=['sketch_id','part','num_strokes'])
    strokes_in_part_df['num_strokes']=pd.to_numeric(strokes_in_part_df['num_strokes'])
    plt.figure()
    b=sns.barplot(x='part',y='num_strokes',data=strokes_in_part_df,ci=95,capsize=0.3, errwidth= 3)
    for item in b.get_xticklabels():
        item.set_rotation(45)

In [ ]:
##Creating a dictionary of sketch_id with associated part sequences
seq_dict={}
for this_sketch in np.unique(stroke_df['sketch_id']):
    parts_list=[]
    DS=stroke_df[stroke_df['sketch_id']==this_sketch]
    for i, row in DS.iterrows():
        parts_list.append(stroke_df['label'][i])
    seq_dict[this_sketch]=parts_list

In [ ]:
##functions for getting 'mean streak_length' from a particular sketch for ground truth and scrambled part orders

import random

def get_mean_streak(sketch_id):
    parts = seq_dict[sketch_id]
    streak_counter=1
    list_of_streaks=[]
    for obj in range(len(parts)-1):
        if parts[obj]==parts[obj+1]:
            streak_counter+=1
        else:
            list_of_streaks.append(streak_counter)
            streak_counter=1 
    list_of_streaks.append(streak_counter)
    return np.mean(list_of_streaks)

def get_scramble_mean_streak(sketch_id):
    parts = seq_dict[sketch_id]
    scram_parts=random.sample(parts,len(parts))
    streak_counter=1
    list_of_streaks=[]
    for obj in range(len(scram_parts)-1):
        if scram_parts[obj]==scram_parts[obj+1]:
            streak_counter+=1
        else:
            list_of_streaks.append(streak_counter)
            streak_counter=1 
    list_of_streaks.append(streak_counter)
    return np.mean(list_of_streaks)

In [ ]:
#Iterating over all sketches to get mean streakiness for each sketch_id

gt_streak_mean={}
for this_cat in unique_cats:
    DS= stroke_df[stroke_df['category']==this_cat]
    streak_mean_list=[]
    for this_sketch in np.unique(DS['sketch_id']):
        streak_mean_list.append(get_mean_streak(this_sketch))
    gt_streak_mean[this_cat]=np.mean(streak_mean_list)

In [ ]:
gt_streak_mean

In [ ]:
streak_diff_dict={}
for this_cat in unique_cats:
    mean_streak_diff_list=[]
    DS=stroke_df[stroke_df['category']==this_cat]
    for i in range(1000):
        scrambled_streaks=[] 
        real_streaks=[]
        for sketch in np.unique(DS['sketch_id']):
            scrambled_streaks.append(get_scramble_mean_streak(sketch))
            real_streaks.append(get_mean_streak(sketch))
        mean_streak_diff_list.append(np.mean(real_streaks)-np.mean(scrambled_streaks))
    streak_diff_dict[this_cat]=mean_streak_diff_list

In [ ]:
len(streak_diff_dict['bird'])

In [ ]:
def CIPlot(category): 
    stroke_df_lite_ss=stroke_df[stroke_df['category']==category]
    mean_streak_diff_list=[]
    for i in range(1000):
        this_round_scrambled_streak=[] 
        this_round_real_streak=[]
        for sketch in np.unique(stroke_df_lite_ss['sketch_id']):
            this_round_real_streak.append(get_mean_streak(sketch))
            this_round_scrambled_streak.append(get_scramble_mean_streak(sketch))
        mean_streak_diff_list.append(np.mean(this_round_real_streak)-np.mean(this_round_scrambled_streak))
    perm_observed_mean_streak_diff = np.mean(mean_streak_diff_list)    
    lb=np.percentile(mean_streak_diff_list,2.5)
    ub=np.percentile(mean_streak_diff_list,97.5)
    plt.figure(figsize=(10,8))
    h=sns.distplot(mean_streak_diff_list,kde=False,hist=True,norm_hist=False)
    plt.axvline(perm_observed_mean_streak_diff, color='yellow', linestyle='solid', linewidth=2)
    plt.axvline(lb, color='orange', linestyle='solid', linewidth=2)
    plt.axvline(ub, color='orange', linestyle='solid', linewidth=2)
    plt.title(category)
    plt.ylabel('count')
    plt.xlabel('streak length difference')
    plt.legend(['mean','95% CI'], ncol=2, bbox_to_anchor=(1, 1.05))
    
    plt.savefig(os.path.join(plot_dir,'Streakiness Diff'),edgecolor='w',bbox_inches='tight')
    plt.show()
    return np.mean(mean_streak_diff_list), np.std(mean_streak_diff_list)




def CIPlotCatCond(category,condition): 
    stroke_df_lite_ss=stroke_df[(stroke_df['category']==category)&(stroke_df['condition']==condition)]
    mean_streak_diff_list=[]
    for i in range(1000):
        this_round_scrambled_streak=[] 
        this_round_real_streak=[]
        for sketch in np.unique(stroke_df_lite_ss['sketch_id']):
            this_round_real_streak.append(get_mean_streak(sketch))
            this_round_scrambled_streak.append(get_scramble_mean_streak(sketch))
        mean_streak_diff_list.append(np.mean(this_round_real_streak)-np.mean(this_round_scrambled_streak))
    perm_observed_mean_streak_diff = np.mean(mean_streak_diff_list)    
    lb=np.percentile(mean_streak_diff_list,2.5)
    ub=np.percentile(mean_streak_diff_list,97.5)
    plt.figure(figsize=(10,8))
    h=sns.distplot(mean_streak_diff_list,kde=False,hist=True,norm_hist=False)
    plt.axvline(perm_observed_mean_streak_diff, color='yellow', linestyle='solid', linewidth=2)
    plt.axvline(lb, color='orange', linestyle='solid', linewidth=2)
    plt.axvline(ub, color='orange', linestyle='solid', linewidth=2)
    plt.title('{}_{}'.format(category,condition))
    plt.ylabel('count')
    plt.xlabel('streak length difference')
    plt.legend(['mean','95% CI'], ncol=2, bbox_to_anchor=(1, 1.05))
    
    plt.savefig(os.path.join(plot_dir,'mean_streak_difference_{}_{}'.format(category, condition)),edgecolor='w',bbox_inches='tight')
    plt.show()
    return perm_observed_mean_streak_diff, lb, ub
    

In [ ]:
for this_cat in unique_cats:
    CIPlot(this_cat)

In [ ]:
for this_condition in np.unique(stroke_df['condition']):
    for this_category in np.unique(stroke_df['category']):
        CIPlotCatCond(this_category, this_condition)

In [ ]:
D_C= D[D['category']=='dog']

In [ ]:
D_C.groupby('label').mean

## Looking at stroke  variation between parts

In [ ]:
###Investigating how stroke length varies between parts given a category of objects 

for category in unique_cats:
    DS= D[D['category']==category]
    num_unique_labs = len(DS['label'].unique())
    strokes_in_part_vect = np.zeros((len(np.unique(DS['annotation_id']))*num_unique_labs,3), dtype='|a1000')
    ind=0
    for this_annotation in np.unique(DS['annotation_id']):    
        DSA= DS[DS['annotation_id']==this_annotation]
        for this_label in np.unique(DS['label']):
            DSB=DSA[DSA['label']==this_label]
            strokes_in_part_vect[ind,]=[this_annotation, this_label,len(np.unique(DSB['stroke_num']))]
            ind+=1
    strokes_in_part_vect=strokes_in_part_vect[~np.all(strokes_in_part_vect == '', axis=1)]
    strokes_in_part_df= pd.DataFrame(strokes_in_part_vect, columns=['annotation_id','part','num_strokes'])
    strokes_in_part_df['num_strokes']=pd.to_numeric(strokes_in_part_df['num_strokes'])
    fig,ax=plt.subplots(1, 1, figsize = (10, 8), dpi=150)
    sns.barplot(x='part',y='num_strokes',data=strokes_in_part_df,ci=95,capsize=0.3, errwidth= 3)
    plt.title('Average number of strokes for {} parts'.format(category))
    plt.ylabel('Number of strokes')
    plt.xlabel('Part')


    
    

In [ ]:
##Investigating how arc length varies between parts

for category in unique_cats:
    plt.figure(figsize=(10,8))
    sns.barplot(x='label', y='arc_length', data=stroke_df[stroke_df['category']=='dog'], ci=68,capsize=0.3, errwidth= 3)
    plt.title('Average stroke lengths for {} parts'.format(category))
    plt.ylabel('arc length')
    plt.xlabel('part')

In [ ]:
###Investigating how arc length varies between parts x conditions

for category in unique_cats:
    plt.figure(figsize=(10,8))
    sns.barplot(x='label',y='arc_length',hue='condition', data=stroke_df[stroke_df['category']==category],ci=68,capsize=0.3, errwidth= 3)
    plt.title('Average stroke lengths for {} parts'.format(category))
    plt.ylabel('Arc length')
    plt.xlabel('Part')

In [ ]:
# ###Might Delete this - calculating average arc length per part manually####
# part_lengths={}
# for category in unique_cats:
#     stroke_df_lite= stroke_df[stroke_df['category']==category]
#     arc_length_dict = OrderedDict()
#     for label in np.unique(stroke_df_lite['label']):
#         stroke_df_lite0= stroke_df_lite[stroke_df_lite['label']==label]
#         path_length=0
#         for stroke in np.unique(stroke_df_lite0['stroke_id']):
#             path = parse_path(stroke_df_lite0[stroke_df_lite['stroke_id']==stroke]['svg'].iloc[0])
#             path_length= path_length+path.length()
#         arc_length_dict[label]= path_length/len(stroke_df_lite0)
#     part_lengths[category]=arc_length_dict


    
    

## Looking at annotation bouts

In [ ]:
### Investigating the number of parts for each part within a category. Does this number correspond to number 
###of strokes with that part label?

In [ ]:
D['part_bout_num'] = pd.to_numeric(D['part_bout_num'])

for category in unique_cats:
    DS=D[D['category']==category]
    unique_labels_in_cat= DS['label'].unique()
    plot_vect = np.zeros([1,2], dtype='|a20')
    for label in unique_labels_in_cat:
        DSA=DS[DS['label']==label]
        total_bouts=0
        for sketch in DSA['sketch_id'].unique():
            DSB= DSA[DSA['sketch_id']==sketch]
            num_bouts = len(DSB['part_bout_num'].unique())
            temp= np.array([label, num_bouts], dtype= '|a20')
            plot_vect=np.vstack((plot_vect, temp))
    plot_vect=np.delete(plot_vect, (0), axis=0)  
    plot_df= pd.DataFrame(plot_vect, columns=['label','num_bouts'])
    plot_df['num_bouts']=pd.to_numeric(plot_df['num_bouts'])
    plt.figure(figsize=(10,8))
    sns.barplot(x='label',y='num_bouts',data= plot_df,capsize=0.3)
    plt.title('Average bout numbers for {} parts'.format(category))
    plt.ylabel('Average Number of Bouts')
    plt.xlabel('Part')

        #label_group_data=DS.groupby('label').agg({'part_bout_num':"mean"})
    

### Creating sketch vectors

In [ ]:
from scipy import stats

In [ ]:
####Creating different vectors for each category, might look at this later

label_vect_dict={}
for cat in unique_cats:
    DS= stroke_df[stroke_df['category']==cat]
    unique_labels_in_cat=valid_labels_dict[cat]
    unique_sketches_in_cat=DS['sketch_id'].unique()
    Label_Vec = np.zeros((len(unique_sketches_in_cat),len(unique_labels_in_cat)*2), dtype=int)
    arc_length_vec = np.zeros((len(unique_sketches_in_cat),len(valid_labels_dict[cat])), dtype=int)
    for s,sketch in enumerate(unique_sketches_in_cat):
        label_vec = np.zeros(len(unique_labels_in_cat),dtype=int)
        arc_vec = np.zeros(len(unique_labels_in_cat),dtype=int)
        DSA=DS[DS['sketch_id']==sketch]
        label_list = DSA.label.values        
        for label in label_list:
            if label in unique_labels_in_cat:
                label_ind = unique_labels_in_cat==label
                label_vec[label_ind] += 1
        for label in unique_labels_in_cat:
            DSB=DSA[DSA['label']==label]
            label_ind = unique_labels_in_cat==label
            arc_vec[label_ind] = DSB['arc_length'].sum()
            
        
        Label_Vec[s,0:len(unique_labels_in_cat)]=label_vec
        Label_Vec[s,len(unique_labels_in_cat):len(unique_labels_in_cat)*2]=arc_vec
    label_vect_dict[cat]= Label_Vec
    

##z scoring

for cat in unique_cats:
    label_vect_dict[cat] = stats.zscore(label_vect_dict[cat], axis=1, ddof=1)
    

In [ ]:
###This is where we make a num unique labels * 2 X number of sketches vector 

feature_vec = np.zeros((len(stroke_df.sketch_id.unique()),len(valid_labels)*2), dtype=int)
ind=0
start_pos=0
end_pos=0
meta_list=[]
cols = ['sketch_id','target','condition','category','outcome']

for cat in unique_cats:
  
    DS= stroke_df[stroke_df['category']==cat]
    unique_labels_in_cat=valid_labels_dict[cat]
    unique_sketches_in_cat=DS['sketch_id'].unique()
    start_pos = end_pos
    end_pos+= len(unique_labels_in_cat)
    print start_pos, end_pos
    Label_Vec = np.zeros((len(unique_sketches_in_cat),len(unique_labels_in_cat)*2), dtype=int)
    arc_length_vec = np.zeros((len(unique_sketches_in_cat),len(valid_labels_dict[cat])), dtype=int)
    for s,sketch in enumerate(unique_sketches_in_cat):
        
        label_vec = np.zeros(len(unique_labels_in_cat),dtype=int)
        arc_vec = np.zeros(len(unique_labels_in_cat),dtype=int)
        DSA=DS[DS['sketch_id']==sketch]
      
        meta_list.append(pd.Series([DSA['sketch_id'],DSA['target'].unique(),DSA['condition'].unique(),DSA['category'].unique(),DSA['outcome'].unique()], index=cols))
        label_list = DSA.label.values        
        for label in label_list:
            if label in unique_labels_in_cat:
                label_ind = unique_labels_in_cat==label
                label_vec[label_ind] += 1
        for label in unique_labels_in_cat:
            DSB=DSA[DSA['label']==label]
            label_ind = unique_labels_in_cat==label
            arc_vec[label_ind] = DSB['arc_length'].sum()
            
        
        feature_vec[ind,start_pos:end_pos]=label_vec
        feature_vec[ind,start_pos+len(valid_labels):end_pos+len(valid_labels)]=arc_vec
        ind+=1
feature_vec_meta = pd.DataFrame(meta_list, columns=cols)

    


    

In [ ]:
##Changing column values from np arrays to strings/boolean

def arr_to_str(arr):
    return (arr[0])
feature_vec_meta['target']=feature_vec_meta['target'].apply(arr_to_str)
feature_vec_meta['condition']=feature_vec_meta['condition'].apply(arr_to_str)
feature_vec_meta['category']=feature_vec_meta['category'].apply(arr_to_str)
feature_vec_meta['outcome']=feature_vec_meta['outcome'].apply(arr_to_str)



In [ ]:
feature_vec.shape

In [ ]:
feature_df= pd.DataFrame(feature_vec, columns=[s + '_numstrokes' for s in valid_labels]+[s + '_total_arclength' for s in valid_labels])

In [ ]:
##creating a compressed version of the feature df with no duplicates for parts

labs_numstrokes=[]
labs_total_arclength=[]
for lab in np.unique(valid_labels):
    labs_numstrokes.append(lab +'_numstrokes')
    labs_total_arclength.append(lab+'_total_arclength')
feature_df_labs=labs_numstrokes+labs_total_arclength   
feature_df_final= pd.DataFrame(columns=feature_df_labs)


for this_lab in feature_df_labs:
    duplicates=[col for col in feature_df if col.startswith(this_lab)]
    feature_df_final[this_lab]= feature_df[duplicates].sum(axis=1)
feature_df = feature_df_final

In [ ]:
##Check to make sure the df looks okay
assert len(feature_df.columns)==len(np.unique(feature_df.columns))
feature_df.head()

In [ ]:
#Normalizing within row within measure (numstrokes/arclength) 

feature_df.iloc[:,0:int(len(feature_df.columns)/2)]=feature_df.iloc[:,0:int(len(feature_df.columns)/2)].div(feature_df.iloc[:,0:int(len(feature_df.columns)/2)].sum(axis=1),axis=0)

feature_df.iloc[:,int(len(feature_df.columns)/2):int(len(feature_df.columns))]=feature_df.iloc[:,int(len(feature_df.columns)/2):int(len(feature_df.columns))].div(feature_df.iloc[:,int(len(feature_df.columns)/2):int(len(feature_df.columns))].sum(axis=1),axis=0)



In [ ]:
###Execute this if we want to save a non-zscore matrix
run=False
if run==True:
    feature_df.to_csv(os.path.join(features_dir,'semantic_parts_sketch_features_compressed_non-whitened.csv'))
run=False    

In [ ]:
#z-scoring within columns

cols=list(feature_df.columns)
for this_col in cols:
    feature_df[this_col]=(feature_df[this_col] - feature_df[this_col].mean())/feature_df[this_col].std(ddof=0)


In [ ]:
###Saving out matrices to csv/npy as needed

In [ ]:
feature_df.to_csv(os.path.join(features_dir,'semantic_parts_sketch_features_compressed.csv'))

In [ ]:
np.save(os.path.join(features_dir, 'semantic_parts_sketch_features'),feature_vec)

In [ ]:
feature_vec_meta.to_csv(os.path.join(csv_dir,'semantic_parts_sketch_features_meta.csv'))

In [ ]:
full_feature_df = pd.concat((feature_vec_meta,feature_df),axis=1)

In [ ]:
full_feature_df

## Some more vector visualization 


### PCA


In [ ]:
from sklearn.decomposition import PCA

In [ ]:
#loading files we need
feature_vec = np.load(os.path.join(features_dir,'semantic_parts_sketch_features.npy'))
feature_df = pd.DataFrame.from_csv(os.path.join(features_dir,'semantic_parts_sketch_features_compressed.csv'))
meta_df= pd.DataFrame.from_csv(os.path.join(features_dir,'semantic_parts_sketch_meta.csv'))


In [ ]:

for this_part in np.unique(valid_labels):
    feature_df.rename(columns={this_part:'{}_numstrokes'.format(this_part)}, inplace=True)

In [ ]:
feature_df

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(feature_df.values)
meta_df['pc_1'] = pca_result[:,0]
meta_df['pc_2'] = pca_result[:,1] 
meta_df['pca_3'] = pca_result[:,2]

In [ ]:
###This chunk may not run because of ggplot being outdate. Will write up vis in matplotlib if necessary

from ggplot import *


cat_plot = ggplot( meta_df, aes(x='pc_1', y='pc_2', color='category') ) \
        + geom_point(size=75,alpha=0.8) \
        + ggtitle("First and Second Principal Components colored by category")
cat_plot





In [ ]:
cond_plot = ggplot( meta_df, aes(x='pc_1', y='pc_2', color='condition') ) \
        + geom_point(size=75,alpha=0.8) \
        + ggtitle("First and Second Principal Components colored by category")
cond_plot

In [ ]:
tar_plot = ggplot( meta_df, aes(x='pc_1', y='pc_2', color='target') ) \
        + geom_point(size=75,alpha=0.8) \
        + ggtitle("First and Second Principal Components colored by category")
tar_plot

### t-SNE

In [ ]:
import time

from sklearn.manifold import TSNE


time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature_df.values)
meta_df['tsne_dim_1']=tsne_results[:,0]
meta_df['tsne_dim_2']=tsne_results[:,1]

In [ ]:

###Visualizing using ggplot. Not as flexible as expected

# plt.figure()
# tsneplot = ggplot( meta_df, aes(x='tsne_dim_1', y='tsne_dim_2', color='category') ) \
#         + geom_point(size=70,alpha=0.1) \
#         + ggtitle("tSNE dimensions colored by category")
# tsneplot
# tsneplot.save(filename = os.path.join(plot_dir,'tSNE dimensions colored by category'))


# plt.figure()
# tsneplot = ggplot( meta_df, aes(x='tsne_dim_1', y='tsne_dim_2', color='condition') ) \
#         + geom_point(size=70,alpha=0.1) \
#         + ggtitle("tSNE dimensions colored by condition")
# tsneplot
# tsneplot.save(filename = os.path.join(plot_dir,'tSNE dimensions colored by condition'))


# tsneplot = ggplot( meta_df, aes(x='tsne_dim_1', y='tsne_dim_2', color='target') ) \
#         + geom_point(size=70,alpha=0.1) \
#         + ggtitle("tSNE dimensions colored by target")\
#         + x_label("die")
# tsneplot
# tsneplot.save(filename = os.path.join(plot_dir,'tSNE dimensions colored by target'))



In [ ]:
##Visualizing t-SNE results

cat_labels = meta_df['category'].unique()
rgb_values = sns.color_palette("Set2", meta_df['category'].nunique())
color_map_cat = dict(zip(cat_labels, rgb_values))

cond_labels = meta_df['condition'].unique()
rgb_values = sns.color_palette("Set2", meta_df['condition'].nunique())
color_map_cond = dict(zip(cond_labels, rgb_values))

target_labels= meta_df['target'].unique()
rgb_values = sns.color_palette("Set2", meta_df['target'].nunique())
color_map_target = dict(zip(target_labels, rgb_values))

f=plt.figure(figsize(10,10))
ax = plt.subplot(aspect='equal')
p= plt.scatter(meta_df['tsne_dim_1'],meta_df['tsne_dim_2'],c=meta_df['category'].map(color_map_cat))
plt.ylabel('TSNE Dim 1')
plt.xlabel('TSNE Dim 2')
plt.tick_params(labelbottom=False, labelleft= False) 
plt.title('TSNE dimensions colored by categories')

txts = []
for cat in enumerate(cat_labels):
        cat=cat[1]
        txt = ax.text(xtext, ytext, str(cat), fontsize=24)
        xtext = np.median(meta_df[meta_df['category'] == cat]['tsne_dim_1'] )      
        ytext = np.median(meta_df[meta_df['category'] == cat]['tsne_dim_2'] )
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])  
        
        txts.append(txt)

f.show
plt.savefig(os.path.join(plot_dir,'TSNE dimensions colored by categories'))





f=plt.figure(figsize(10,10))
ax = plt.subplot(aspect='equal')
plt.scatter(meta_df['tsne_dim_1'],meta_df['tsne_dim_2'],c=meta_df['condition'].map(color_map_cond))
plt.ylabel('TSNE Dim 1')
plt.xlabel('TSNE Dim 2')
plt.tick_params(labelbottom=False, labelleft= False)   
plt.title('TSNE dimensions colored by condition')
f.show
plt.savefig(os.path.join(plot_dir,'TSNE dimensions colored by condition'))







f=plt.figure(figsize(10,10))
plt.scatter(meta_df['tsne_dim_1'],meta_df['tsne_dim_2'],c=meta_df['target'].map(color_map_target))
plt.ylabel('TSNE Dim 1')
plt.xlabel('TSNE Dim 2')
plt.tick_params(labelbottom=False, labelleft= False) 
plt.title('TSNE dimensions colored by target')

f.show
plt.savefig(os.path.join(plot_dir,'TSNE dimensions colored by target'))